# Bare metal x86_64 Assembly Language on *nix 64

Using the NASM assembler (https://nasm.us/)

To install on Fedora: # dnf -y install nasm

* SYSCALL instruction jumps to operating system services. To use, first put the system call number in RAX, then the arguments, if any, in RDI, RSI, RDX, R10, R8, and R9, respectively. 
* call number 60 exit a process.

## Hello world example

### Write the source file

In [4]:
%%writefile hello.asm
; hello.asm
; source: https://cs.lmu.edu/~ray/notes/x86assembly/
            global      _start
            section     .text
_start:     mov         rax, 1          ; system call for write
            mov         rdi, 1          ; file handle 1 is stdout
            mov         rsi, message    ; address of string to output
            mov         rdx, 14         ; number of bytes
            syscall                     ; invoke operating system to do the write
            mov         rax, 60         ; system call for exit
            xor         rdi, rdi        ; exit code 0
            syscall                     ; invoke operating system to exit

            section     .data
message:    db          "Hello, world!", 10   ; note the newline (10) at the end

Overwriting hello.asm


### Assemble and show the listing file

In [5]:
! nasm -f elf64 hello.asm -l hello.lst

In [6]:
! cat hello.lst

     1                                  ; hello.asm
     2                                  ; source: https://cs.lmu.edu/~ray/notes/x86assembly/
     3                                              global      _start
     4                                              section     .text
     5 00000000 B801000000              _start:     mov         rax, 1          ; system call for write
     6 00000005 BF01000000                          mov         rdi, 1          ; file handle 1 is stdout
     7 0000000A 48BE-                               mov         rsi, message    ; address of string to output
     7 0000000C [0000000000000000] 
     8 00000014 BA0E000000                          mov         rdx, 14         ; number of bytes
     9 00000019 0F05                                syscall                     ; invoke operating system to do the write
    10 0000001B B83C000000                          mov         rax, 60         ; system call for exit
    11 00000020 4831FF             

Columns, in sequence:
* sequential line number, only for reference
* address in memory
* bytes (data or instruction)
* label
* opcode
* operand, register, data, other information
* comments

### Link

In [7]:
! ld --verbose -o hello.out hello.o

GNU ld version 2.35-18.fc33
  Supported emulations:
   elf_x86_64
   elf32_x86_64
   elf_i386
   elf_iamcu
   elf_l1om
   elf_k1om
   i386pep
   i386pe
   elf64bpf
using internal linker script:
/* Script for -z combreloc -z separate-code */
/* Copyright (C) 2014-2020 Free Software Foundation, Inc.
   Copying and distribution of this script, with or without modification,
   are permitted in any medium without royalty provided the copyright
   notice and this notice are preserved.  */
OUTPUT_FORMAT("elf64-x86-64", "elf64-x86-64",
	      "elf64-x86-64")
OUTPUT_ARCH(i386:x86-64)
ENTRY(_start)
SEARCH_DIR("=/usr/x86_64-redhat-linux/lib64"); SEARCH_DIR("=/usr/lib64"); SEARCH_DIR("=/usr/local/lib64"); SEARCH_DIR("=/lib64"); SEARCH_DIR("=/usr/x86_64-redhat-linux/lib"); SEARCH_DIR("=/usr/local/lib"); SEARCH_DIR("=/lib"); SEARCH_DIR("=/usr/lib");
SECTIONS
{
  PROVIDE (__executable_start = SEGMENT_START("text-segment", 0x400000)); . = SEGMENT_START("text-segment", 0x400000) + SIZEOF_HEADERS;
  .in

### Show generated files

In [8]:
! ls

assembly.ipynb		hello.f90		  hello.lst  hello.s
fortran-assembly.ipynb	hello.f90.004t.original   hello.o    README.md
hello.asm		hello.f90.235t.optimized  hello.out


### Run

In [9]:
! ./hello.out

Hello, world!


### readelf

In [2]:
! readelf -all hello.o

ELF Header:
  Magic:   7f 45 4c 46 02 01 01 00 00 00 00 00 00 00 00 00 
  Class:                             ELF64
  Data:                              2's complement, little endian
  Version:                           1 (current)
  OS/ABI:                            UNIX - System V
  ABI Version:                       0
  Type:                              REL (Relocatable file)
  Machine:                           Advanced Micro Devices X86-64
  Version:                           0x1
  Entry point address:               0x0
  Start of program headers:          0 (bytes into file)
  Start of section headers:          64 (bytes into file)
  Flags:                             0x0
  Size of this header:               64 (bytes)
  Size of program headers:           0 (bytes)
  Number of program headers:         0
  Size of section headers:           64 (bytes)
  Number of section headers:         7
  Section header string table index: 3

Section Headers:
  [Nr] Name              Type     

### read the .data section in hex

In [4]:
! readelf -x '.data'  hello.o


Hex dump of section '.data':
  0x00000000 48656c6c 6f2c2077 6f726c64 210a     Hello, world!.



### objdump

In [6]:
! objdump -d hello.o


hello.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <_start>:
   0:	b8 01 00 00 00       	mov    $0x1,%eax
   5:	bf 01 00 00 00       	mov    $0x1,%edi
   a:	48 be 00 00 00 00 00 	movabs $0x0,%rsi
  11:	00 00 00 
  14:	ba 0e 00 00 00       	mov    $0xe,%edx
  19:	0f 05                	syscall 
  1b:	b8 3c 00 00 00       	mov    $0x3c,%eax
  20:	48 31 ff             	xor    %rdi,%rdi
  23:	0f 05                	syscall 


## asm + C library

In [11]:
%%writefile hc.asm
    global  main
    extern  puts

    section .text
main:                     ; This is called by the C library startup code
    mov     rdi, message  ; First integer|pointer argument in rdi
    call    puts          ; puts(message)
    ret                   ; Return from main back into C library wrapper
message:
    db      "Hello, world!", 0    ; strings must be terminated with 0

Overwriting hc.asm


### Assemble using NASM

In [12]:
! nasm -f elf64 hc.asm -l hc.lst

In [13]:
! cat hc.lst

     1                                      global  main
     2                                      extern  puts
     3                                  
     4                                      section .text
     5                                  main:                     ; This is called by the C library startup code
     6 00000000 48BF-                       mov     rdi, message  ; First integer|pointer argument in rdi
     6 00000002 [1000000000000000] 
     7 0000000A E8(00000000)                call    puts          ; puts(message)
     8 0000000F C3                          ret                   ; Return from main back into C library wrapper
     9                                  message:
    10 00000010 48656C6C6F2C20776F-         db      "Hello, world!", 0    ; strings must be terminated with 0
    10 00000019 726C642100         


### Link using gcc

In [15]:
! gcc -o hc.out hc.o

### Run

In [16]:
! ./hc.out

Hello, world!


# asm & f90

Reference: https://forum.nasm.us/index.php?topic=2448.15

In [83]:
%%writefile hcfg.asm
            global      hello_
            section     .text
hello_:     mov         rax, 1          ; system call for write
            mov         rdi, 1          ; file handle 1 is stdout
            mov         rsi, message    ; address of string to output
            mov         rdx, 20         ; number of bytes
            syscall                     ; invoke operating system to do the write
            mov         rax, 60         ; system call for exit
            xor         rdi, rdi        ; exit code 0
            syscall                     ; invoke operating system to exit

            section     .data
message:    db          " asm: Hello, world!", 10   ; note the newline (10) at the end

Overwriting hcfg.asm


In [84]:
! nasm -f elf64 hcfg.asm

In [85]:
%%writefile hcfg.f90
external hello
print*, "f90: Hello, world!"
i = hello()
end

Overwriting hcfg.f90


In [86]:
! gfortran -o hcfg.out hcfg.f90 hcfg.o

In [87]:
! ./hcfg.out

 f90: Hello, world!
 asm: Hello, world!
